### Notebook for concept detection in neural network

In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
import numpy as np
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

from concepts import static_concepts, linear_regression, generate_static_concept_datasets
from policy import ConvNet, ResNet
from utils import concept_folder_setup_and_score

import tensorflow as tf

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

model_name = "net"
session_name = "falcon"
board_size = 7
board_name = f'{board_size}x{board_size}'
resnet = True

model_type = "resnet" if resnet else "convnet"

agents_to_sample = [0, 10, 20, 60, 100, 500]

full_model_path = f"../models/saved_sessions/{model_type}/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = static_concepts.one_eye

CONCEPT_NAME = static_concepts.one_eye.__name__

CASES_TO_SAMPLE = 100 # 25000


In [15]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, model_path)
    else:
        model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [16]:
positive_cases, negative_cases = generate_static_concept_datasets(CASES_TO_SAMPLE, agents, board_size, CONCEPT_FUNC)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases: 441it [00:32, 13.39it/s]                        


In [6]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (100, 5, 7, 7)
Negative cases:  (100, 5, 7, 7)


In [7]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [7]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  3


In [ ]:
from env import gogame
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [18]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True
epochs = 100

Positions to consider: 160


In [19]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY,
    epochs=epochs
)

score = 0 if score < 0 else score

print("Regression score: ", score)

concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, name, score)

Epoch 1/100


2024-02-19 11:34:13.725667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-19 11:34:13.758206: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


3/5 [=================>............] - ETA: 0s - loss: 0.9097

2024-02-19 11:34:14.434704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 140ms/step - loss: 0.9207 - val_loss: 0.9252
Epoch 2/100
5/5 [==============================] - 0s 42ms/step - loss: 0.9173 - val_loss: 0.9234
Epoch 3/100
5/5 [==============================] - 0s 32ms/step - loss: 0.9142 - val_loss: 0.9216
Epoch 4/100
5/5 [==============================] - 0s 33ms/step - loss: 0.9110 - val_loss: 0.9201
Epoch 5/100
5/5 [==============================] - 0s 32ms/step - loss: 0.9085 - val_loss: 0.9187
Epoch 6/100
5/5 [==============================] - 0s 28ms/step - loss: 0.9053 - val_loss: 0.9172
Epoch 7/100
5/5 [==============================] - 0s 28ms/step - loss: 0.9026 - val_loss: 0.9158
Epoch 8/100
5/5 [==============================] - 0s 31ms/step - loss: 0.8999 - val_loss: 0.9144
Epoch 9/100
5/5 [==============================] - 0s 29ms/step - loss: 0.8974 - val_loss: 0.9130
Epoch 10/100
5/5 [==============================] - 0s 30ms/step - loss: 0.8950 - val_loss: 0.9116
Epoch 11/100
5/5 [============

2024-02-19 11:34:28.297241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-19 11:34:28.398942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [20]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=epochs
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
        
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, epoch, concept_presence_per_layer)

Getting activation outputs: 100%|██████████| 7/7 [00:00<00:00, 31.60it/s]


Performing regression for layer 0
Epoch 1/100


2024-02-19 11:34:59.473155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-19 11:34:59.499739: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


4/5 [=======================>......] - ETA: 0s - loss: 1.2816

2024-02-19 11:35:00.042617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

2024-02-19 11:46:21.050447: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-19 11:46:21.074411: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


4/5 [=======================>......] - ETA: 0s - loss: 0.8844

2024-02-19 11:46:21.739110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 154ms/step - loss: 0.8680 - val_loss: 0.8603
Epoch 2/100
5/5 [==============================] - 0s 40ms/step - loss: 0.8670 - val_loss: 0.8594
Epoch 3/100
5/5 [==============================] - 0s 36ms/step - loss: 0.8659 - val_loss: 0.8585
Epoch 4/100
5/5 [==============================] - 0s 32ms/step - loss: 0.8650 - val_loss: 0.8576
Epoch 5/100
5/5 [==============================] - 0s 32ms/step - loss: 0.8640 - val_loss: 0.8568
Epoch 6/100
5/5 [==============================] - 0s 30ms/step - loss: 0.8631 - val_loss: 0.8559
Epoch 7/100
5/5 [==============================] - 0s 29ms/step - loss: 0.8624 - val_loss: 0.8550
Epoch 8/100
5/5 [==============================] - 0s 29ms/step - loss: 0.8613 - val_loss: 0.8541
Epoch 9/100
5/5 [==============================] - 0s 28ms/step - loss: 0.8604 - val_loss: 0.8533
Epoch 10/100
5/5 [==============================] - 0s 30ms/step - loss: 0.8595 - val_loss: 0.8524
Epoch 11/100
5/5 [============

2024-02-19 11:46:35.965503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-19 11:46:36.098744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
